In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00


In [ ]:
import torch
import re
import pandas as pd
from collections import Counter
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Initialize model and tokenizer
# try:
#     tokenizer = AutoTokenizer.from_pretrained("/content/my_model")
#     model = AutoModelForMaskedLM.from_pretrained("/content/my_model").to(device)
#     print("Loaded fine-tuned model from /content/my_model")
# except:
print("Using pretrained model")
model_name = "UBC-NLP/MARBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)

Using pretrained model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Data preprocessing functions**

In [ ]:
def preprocess(sentence: str) -> str:
    """Enhanced Arabic text preprocessing"""
    sentence = sentence.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا')
    arabic_chars = (
        'ابتثجحخدذرزسشصضطظعغفقكلمنهويءئؤىة'
        'ًٌٍَُِّْٰ'
        '0123456789'
        '.,!؟؛،'
    )
    return ''.join([c for c in sentence if c in arabic_chars or c.isspace()])


def data_vocab(dataframe, min_freq=3):
    """Create vocabulary with frequency filtering"""
    words_freq = Counter()
    for text in dataframe['text']:
        words_freq.update(text.split())
    return {word: freq for word, freq in words_freq.items() if freq >= min_freq}


def tokenize_function(examples):
    """Tokenization function for dataset"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

# **Prediction functions**

In [ ]:
def find_misspellings(text: str, vocab: dict, threshold: float = 0.15) -> list:
    """Identify potentially misspelled words using MLM probability and vocab"""
    words = text.split()
    misspelled_indices = []

    for i, word in enumerate(words):
        if word not in vocab:
            masked_words = words.copy()
            masked_words[i] = tokenizer.mask_token
            masked_sentence = " ".join(masked_words)

            inputs = tokenizer(masked_sentence, return_tensors="pt").to(model.device)
            mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits[0, mask_token_index]
                probs = torch.softmax(logits, dim=-1).squeeze()
                word_id = tokenizer.encode(word, add_special_tokens=False)
                word_prob = torch.mean(probs[word_id]) if word_id else 0

            if word_prob < threshold:
                misspelled_indices.append(i)

    return misspelled_indices


def generate_masked_sentences(text: str, misspelled_indices: list) -> list:
    """Generate masked sentences for each misspelled word"""
    words = text.split()
    return [
        " ".join(words[:idx] + [tokenizer.mask_token] + words[idx + 1:])
        for idx in misspelled_indices
    ]


def predict(masked_sentence: str, top_k=5) -> str:
    """Predict masked word in sentence using MLM"""
    inputs = tokenizer(masked_sentence, return_tensors="pt").to(device)
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits[0, mask_token_index]
    probs = torch.softmax(logits, dim=-1).squeeze()
    top_k_tokens = torch.topk(probs, top_k)

    for token_id in top_k_tokens.indices:
        token = tokenizer.decode([token_id]).strip()
        if re.match(r'^[\u0600-\u06FF]{2,}$', token):
            return token

    return "[UNK]"

In [ ]:
test_cases = [
    ("وززارة [MASK] والتعليم", "التربية"),  # Should predict "التربية"
    ("يوم [MASK]", "السبت")               # Should predict "السبت"
]
for sentence, expected in test_cases:
    pred = predict(sentence)
    print(f"Model {'✅' if pred == expected else '❌'} {sentence} → {pred} (Expected: {expected})")

Model ✅ وززارة [MASK] والتعليم → التربية (Expected: التربية)
Model ❌ يوم [MASK] → جميل (Expected: السبت)


# **Pipeline function**

In [ ]:
def pipeline(input_text: str, verbose: bool = True) -> str:
    """
    Full pipeline for Arabic spelling correction.
    """
    processed_text = preprocess(input_text)
    vocab = data_vocab(df, min_freq=3)
    misspelled_indices = find_misspellings(processed_text, vocab)

    if not misspelled_indices:
        if verbose:
            print("✅ لا توجد أخطاء إملائية واضحة.")
        return processed_text

    masked_sentences = generate_masked_sentences(processed_text, misspelled_indices)
    words = processed_text.split()
    corrections = {}

    for idx, masked in zip(misspelled_indices, masked_sentences):
        correction = predict(masked)
        corrections[words[idx]] = correction
        words[idx] = correction

    corrected_sentence = " ".join(words)

    if verbose:
        print("🔍 الكلمات التي تم تصحيحها:")
        for original, corrected in corrections.items():
            print(f" - {original} ➤ {corrected}")

    return corrected_sentence

In [ ]:
# Load and prepare dataset
df = pd.read_csv('/content/drive/MyDrive/arabic_dataset_classifiction.csv')
print(df)

                                                     text  targe
0       بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...      0
1       قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...      0
2       أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...      0
3       اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...      0
4       تزال صناعة الجلود في المغرب تتبع الطريقة التقل...      0
...                                                   ...    ...
111723  اللاعب تأخر في العودة إلى التداريب والمدرب غاض...      4
111724  المشرف العام لحسنية أكادير قال إنه سيغادر الفر...      4
111725  نسب إليه نتائج الوداد وصحوة الرجاء وآخر صيحاته...      4
111726  ستحتضن الرباط في الفترة مابين يوليوز المقبل دو...      4
111727  رضخ أحمد أهمو رئيس أمل تيزنيت لكرة القدم لضغوط...      4

[111728 rows x 2 columns]


In [ ]:
# Clean dataset
df = df.drop(columns=['targe'], errors='ignore').dropna().drop_duplicates()
df['text'] = df['text'].apply(preprocess)
df['text'] = df['text'].apply(lambda x: x if len(x.split()) > 5 else None)
df = df.dropna().reset_index(drop=True)

In [ ]:
# Create vocabulary
words_freq = data_vocab(df)

In [ ]:
# Prepare Hugging Face dataset
dataset = Dataset.from_pandas(df[:10000])  #  10,000 samples
dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Training configuration
training_args = TrainingArguments(
    output_dir="./model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=7,  #  7 epochs
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    fp16=True,
    save_strategy="epoch",
    logging_steps=50,
    optim="adafactor"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-18-d34e912acbdb>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train and save model
trainer.train()
trainer.save_model("/content/my_model")
tokenizer.save_pretrained("/content/my_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: housam3a (housam3a-helwan-universty) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.417700,3.165819
2,3.092100,3.001456
3,2.941300,2.933681
4,2.815600,2.906623
5,2.680100,2.820498
6,2.622400,2.774557
7,2.532600,2.733945


('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/vocab.txt',
 '/content/my_model/added_tokens.json',
 '/content/my_model/tokenizer.json')

In [ ]:
input_texts = [
    "وززارة النربية والتعليم تعلن عن تعطبل الدراسة رسميا يوم السسبت نظرا للظروف الجوية الحالييه وحفاظا على سلامة المعلمون و الططلاب",
    "بعد أن قرر القضاء الفنسي الإبقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الحميس بتهمتي القتتل و التحرش",
    "قال الفنان المغربي محمد الخياري نحن احرار الامه الاسلاميه و لن نسمح بحدوث حربب في الوطن"
]

for input_text in input_texts:
    true_sentence = pipeline(input_text)
    print('Incorrect Sentence:', input_text)
    print('Corrected Sentence:', true_sentence)
    print('-' * 20)  # Add a separator for clarity

🔍 الكلمات التي تم تصحيحها:
 - وززارة ➤ وزارة
 - النربية ➤ التربية
 - تعطبل ➤ تعليق
 - السسبت ➤ غد
 - الحالييه ➤ السايدة
 - المعلمون ➤ الطلبة
 - الططلاب ➤ الطلبة
Incorrect Sentence: وززارة النربية والتعليم تعلن عن تعطبل الدراسة رسميا يوم السسبت نظرا للظروف الجوية الحالييه وحفاظا على سلامة المعلمون و الططلاب
Corrected Sentence: وزارة التربية والتعليم تعلن عن تعليق الدراسة رسميا يوم غد نظرا للظروف الجوية السايدة وحفاظا على سلامة الطلبة و الطلبة
--------------------
🔍 الكلمات التي تم تصحيحها:
 - الفنسي ➤ الفرنسي
 - الحميس ➤ امس
 - القتتل ➤ الاغتصاب
Incorrect Sentence: بعد أن قرر القضاء الفنسي الإبقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الحميس بتهمتي القتتل و التحرش
Corrected Sentence: بعد ان قرر القضاء الفرنسي الابقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم امس بتهمتي الاغتصاب و التحرش
--------------------
🔍 الكلمات التي تم تصحيحها:
 - الاسلاميه ➤ العربية
 - حربب ➤ فوضى
Incorrect Sentence: قال الفنان المغربي محمد الخياري نحن احرار الامه الاسلاميه و لن نسمح بحدوث حربب في ا

In [ ]:
# Save model and code for local use
import os
import pickle

# Create directory for the project
os.makedirs("arabic_autocorrect", exist_ok=True)

# Save the model and tokenizer
model_path = "arabic_autocorrect/model"
os.makedirs(model_path, exist_ok=True)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Create a Python file with all the code
with open("arabic_autocorrect/autocorrect.py", "w", encoding="utf-8") as f:
    f.write("""import torch
import re
import pandas as pd
from collections import Counter
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model and tokenizer
model_path = "model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path).to(device)

def preprocess(sentence: str) -> str:
    \"\"\"Enhanced Arabic text preprocessing\"\"\"
    sentence = sentence.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا')
    arabic_chars = (
        'ابتثجحخدذرزسشصضطظعغفقكلمنهويءئؤىة'
        'ًٌٍَُِّْٰ'
        '0123456789'
        '.,!؟؛،'
    )
    return ''.join([c for c in sentence if c in arabic_chars or c.isspace()])

def data_vocab(dataframe, min_freq=3):
    \"\"\"Create vocabulary with frequency filtering\"\"\"
    words_freq = Counter()
    for text in dataframe['text']:
        words_freq.update(text.split())
    return {word: freq for word, freq in words_freq.items() if freq >= min_freq}

def find_misspellings(text: str, vocab: dict, threshold: float = 0.15) -> list:
    \"\"\"Identify potentially misspelled words using MLM probability and vocab\"\"\"
    words = text.split()
    misspelled_indices = []

    for i, word in enumerate(words):
        if word not in vocab:
            masked_words = words.copy()
            masked_words[i] = tokenizer.mask_token
            masked_sentence = " ".join(masked_words)

            inputs = tokenizer(masked_sentence, return_tensors="pt").to(model.device)
            mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits[0, mask_token_index]
                probs = torch.softmax(logits, dim=-1).squeeze()
                word_id = tokenizer.encode(word, add_special_tokens=False)
                word_prob = torch.mean(probs[word_id]) if word_id else 0

            if word_prob < threshold:
                misspelled_indices.append(i)

    return misspelled_indices

def generate_masked_sentences(text: str, misspelled_indices: list) -> list:
    \"\"\"Generate masked sentences for each misspelled word\"\"\"
    words = text.split()
    return [
        " ".join(words[:idx] + [tokenizer.mask_token] + words[idx + 1:])
        for idx in misspelled_indices
    ]

def predict(masked_sentence: str, top_k=5) -> str:
    \"\"\"Predict masked word in sentence using MLM\"\"\"
    inputs = tokenizer(masked_sentence, return_tensors="pt").to(device)
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits[0, mask_token_index]
    probs = torch.softmax(logits, dim=-1).squeeze()
    top_k_tokens = torch.topk(probs, top_k)

    for token_id in top_k_tokens.indices:
        token = tokenizer.decode([token_id]).strip()
        if re.match(r'^[\\u0600-\\u06FF]{2,}$', token):
            return token

    return "[UNK]"

def pipeline(input_text: str, verbose: bool = True) -> str:
    \"\"\"
    Full pipeline for Arabic spelling correction.
    \"\"\"
    processed_text = preprocess(input_text)

    # Load vocabulary (you may want to save/load this from a file for production)
    try:
        import pickle
        with open('vocab.pkl', 'rb') as f:
            vocab = pickle.load(f)
    except:
        print("Warning: No vocabulary file found. Using empty vocabulary.")
        vocab = {}

    misspelled_indices = find_misspellings(processed_text, vocab)

    if not misspelled_indices:
        if verbose:
            print("✅ لا توجد أخطاء إملائية واضحة.")
        return processed_text

    masked_sentences = generate_masked_sentences(processed_text, misspelled_indices)
    words = processed_text.split()
    corrections = {}

    for idx, masked in zip(misspelled_indices, masked_sentences):
        correction = predict(masked)
        corrections[words[idx]] = correction
        words[idx] = correction

    corrected_sentence = " ".join(words)

    if verbose:
        print("🔍 الكلمات التي تم تصحيحها:")
        for original, corrected in corrections.items():
            print(f" - {original} ➤ {corrected}")

    return corrected_sentence

if __name__ == "__main__":
    # Example usage
    input_text = input("Enter Arabic text to correct: ")
    corrected = pipeline(input_text)
    print("Corrected text:", corrected)""")

# Create a requirements.txt file
with open("arabic_autocorrect/requirements.txt", "w") as f:
    f.write("""torch>=1.7.0
transformers>=4.5.0
pandas>=1.0.0
tqdm>=4.45.0""")

# Create a README file with separate write statements to avoid triple quote issues
with open("arabic_autocorrect/README.md", "w", encoding="utf-8") as f:
    f.write("# Arabic Text Autocorrection\n\n")
    f.write("This project provides an Arabic text autocorrection system using a masked language model approach.\n\n")
    f.write("## Setup\n\n")
    f.write("1. Install the required packages:\n")
    f.write("   ```\n   pip install -r requirements.txt\n   ```\n\n")
    f.write("2. Make sure the model files are in the `model` directory.\n\n")
    f.write("## Usage\n\n")
    f.write("Run the autocorrection script:\n")
    f.write("```\npython autocorrect.py\n```\n\n")
    f.write("You can also import the functions in your own code:\n")
    f.write("```python\nfrom autocorrect import pipeline\n\n")
    f.write("corrected_text = pipeline(\"وززارة النربية والتعليم\")\n")
    f.write("print(corrected_text)  # Should print \"وزارة التربية والتعليم\"\n```")

# Save the vocabulary for local use
with open("arabic_autocorrect/vocab.pkl", "wb") as f:
    pickle.dump(words_freq, f)

print("Project saved to 'arabic_autocorrect' directory.")
print("You can now copy this directory to your local machine and run it in VS Code.")

Project saved to 'arabic_autocorrect' directory.
You can now copy this directory to your local machine and run it in VS Code.


In [ ]:
   # Create a zip file of the project
   !zip -r arabic_autocorrect.zip arabic_autocorrect/


  adding: arabic_autocorrect/ (stored 0%)
  adding: arabic_autocorrect/vocab.pkl (deflated 62%)
  adding: arabic_autocorrect/requirements.txt (deflated 12%)
  adding: arabic_autocorrect/model/ (stored 0%)
  adding: arabic_autocorrect/model/special_tokens_map.json (deflated 42%)
  adding: arabic_autocorrect/model/vocab.txt (deflated 61%)
  adding: arabic_autocorrect/model/tokenizer_config.json (deflated 74%)
  adding: arabic_autocorrect/model/model.safetensors (deflated 7%)
  adding: arabic_autocorrect/model/config.json (deflated 53%)
  adding: arabic_autocorrect/model/tokenizer.json (deflated 72%)
  adding: arabic_autocorrect/README.md (deflated 42%)
  adding: arabic_autocorrect/autocorrect.py (deflated 61%)
